In [1]:
#New file to vectorise the reviews to perform the data science on them

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math


<b><h2>Handling of the review data and vectorising of the text

In [3]:
full_data = pd.read_csv("../data/review_dataset.csv")
full_data = pd.DataFrame(full_data)
full_data.dropna()
full_data.drop("Unnamed: 0", axis=1)

,Business Name,Street Number,Route,Postal Town,County,Country,Postcode,Overall Rating,Language,Rating,Review Description,Time,Date,processed review,lemmatized review,Keyword Contains,Keyword No Negative
0,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,have to say this is the best salon ive ever be...,1.652418e+09,2022-05-13 05:07:20,say best salon ive ever 4 weeks lifting acryli...,say best salon ive ever 4 week lifting acryli...,False,False
1,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,nian is incredible ive been coming here for a ...,1.661441e+09,2022-08-25 15:22:45,nian incredible ive coming couple months every...,nian incredible ive coming couple month every...,False,False
2,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,i got a new set of gels from tee who was so so...,1.658412e+09,2022-07-21 14:04:52,got new set gels tee lovely chat helped choose...,got new set gel tee lovely chat helped choose...,False,False
3,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,i have been coming to this salon since i moved...,1.660734e+09,2022-08-17 10:57:00,coming salon since moved birmingham took find ...,coming salon since moved birmingham took find...,False,False
4,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,i had my nails done for the first time and i c...,1.659646e+09,2022-08-04 20:41:01,nails done first time say place super cozy gir...,nail done first time say place super cozy gir...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22374,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,just had ford ranger polished so pleased with ...,1.654364e+09,2022-06-04 17:31:51,ford ranger polished pleased results vehicle l...,ford ranger polished pleased result vehicle l...,False,False
22375,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,the name says it all fantastic service great p...,1.641225e+09,2022-01-03 15:44:22,name says fantastic service great prices top q...,name say fantastic service great price top qu...,False,False
22376,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,a brilliant super hand car wash drove away spa...,1.652984e+09,2022-05-19 18:06:27,brilliant super hand car wash drove away spark...,brilliant super hand car wash drove away spar...,False,False
22377,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,always happy when my black and yellow fiesta c...,1.654781e+09,2022-06-09 13:28:00,always happy black yellow fiesta comes gleamin...,always happy black yellow fiesta come gleamin...,False,False


In [4]:
from sklearn.model_selection import train_test_split


In [5]:
X_train_text, X_test_text = train_test_split(full_data,test_size=0.33)
y = X_train_text["Keyword Contains"].copy()
test_y = X_test_text["Keyword Contains"].copy()
X_train_text.drop("Keyword Contains",axis=1)
X_test_text.drop("Keyword Contains",axis=1)

,Unnamed: 0,Business Name,Street Number,Route,Postal Town,County,Country,Postcode,Overall Rating,Language,Rating,Review Description,Time,Date,processed review,lemmatized review,Keyword No Negative
3900,3900,Newry Beauty Centre,32,Upper Water Street,Newry,"Newry, Mourne and Down",United Kingdom,BT35 6AA,5.0,en,5.0,monika is the best nail technician in the area...,1.497171e+09,2017-06-11 08:57:36,monika best nail technician area great price w...,monika best nail technician area great price ...,False
21260,21260,Hand Car Wash,1163,Burbury Street,Birmingham,West Midlands,United Kingdom,B29 7RP,4.4,en,5.0,best car wash in birmingham,1.651149e+09,2022-04-28 12:32:15,best car wash birmingham,best car wash birmingham,False
20867,20867,Phoenix Clifton nails and Beauty,356,Southchurch Drive,Nottingham,Nottingham,United Kingdom,NG11 9FE,4.4,en,1.0,was not happy with how my toe nails came out a...,1.653738e+09,2022-05-28 11:31:47,happy toe nails came told colour chose colour ...,happy toe nail came told colour chose colour ...,False
13766,13766,Super hand car wash,3-7,Ivy Road,Birmingham,West Midlands,United Kingdom,B21 9BP,4.2,en,1.0,i was so disappointed with service they made m...,1.646780e+09,2022-03-08 22:59:56,disappointed service made wait long time wery ...,disappointed service made wait long time wery...,False
5424,5424,Infinity Nails,44,Withybush Road,Haverfordwest,Pembrokeshire,United Kingdom,SA62 4BS,3.8,en,5.0,amazing staff who did a brilliant job my nails...,1.515355e+09,2018-01-07 20:03:51,amazing staff brilliant job nails lasted long ...,amazing staff brilliant job nail lasted long ...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17522,17522,The Car Wash Company Stirling (The Thistles Sh...,11,Saint Davids Way,Stirling,Stirling,United Kingdom,FK8 2EA,2.7,en,3.0,friendly staff however the manager immediately...,1.654973e+09,2022-06-11 18:43:58,friendly staff however manager immediately rep...,friendly staff however manager immediately re...,False
2815,2815,Wisunya's Nail Bar,25,Albion Place,Leeds,West Yorkshire,United Kingdom,LS1 6JS,4.6,en,5.0,ive been numerous times now for my nails done ...,1.647989e+09,2022-03-22 22:40:55,ive numerous times nails done always extremely...,ive numerous time nail done always extremely ...,False
1888,1888,Cosmetology Dundee,122,King Street,Dundee,Dundee City Council,United Kingdom,DD5 1EW,5.0,en,5.0,i love coming to cosmetology and my nails alwa...,1.633101e+09,2021-10-01 15:04:56,love coming cosmetology nails always look flaw...,love coming cosmetology nail always look flaw...,False
18889,18889,Platinum Estates,3,George Street,Derby,Derby,United Kingdom,DE1 1EH,3.5,en,5.0,i have had my property under the fully managed...,1.634820e+09,2021-10-21 12:46:58,property fully managed service platinum estate...,property fully managed service platinum estat...,False


In [7]:
count = 0
for i in X_train_text["Keyword Contains"]:
    if i == True:
        count += 1

print((count/len(X_train_text["Review Description"]))*100)

count = 0
for i in X_test_text["Keyword Contains"]:
    if i == True:
        count += 1

print((count/len(X_test_text["Review Description"]))*100)

3.9685186420329486
3.8180341186027618


<h3><b>Sklearn.feature_extraction attempt to vectorise the data



In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD

In [9]:
vectoriser = TfidfVectorizer(ngram_range=(1,5))
train_tfidf = vectoriser.fit_transform(X_train_text["lemmatized review"])
test_tfidf = vectoriser.transform(X_test_text["lemmatized review"])

# full_text = vectoriser.fit_transform(full_data['lemmatized review'])
# features = vectoriser.get_feature_names_out()

In [10]:
print("Train Shape: {} Rows and {} Columns".format(*train_tfidf.shape))
print("Test Shape: {} Rows and {} Columns".format(*test_tfidf.shape))

Train Shape: 14993 Rows and 1381102 Columns
Test Shape: 7386 Rows and 1381102 Columns


In [11]:
svd = TruncatedSVD()
svd.fit(train_tfidf)

TruncatedSVD()

In [12]:
X = svd.transform(train_tfidf)
test_tfidf = svd.transform(test_tfidf)

In [13]:
model = LogisticRegression()
model.fit(X,y)

LogisticRegression()

In [14]:
submission = model.predict(test_tfidf)
submission_df = pd.Series(submission).rename("Predicted Exploitation")
